In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import scipy.stats as st

# Group model

In [4]:
Perfomance_1 = pd.read_csv('Performance_0906_Inputs.csv')
Perfomance_1['Feature_Extract'] = 'RestNet50'

In [5]:
Perfomance_2 = pd.read_csv('Performance_0907_Feature_extract.csv')
Perfomance_2['Inputs'] = 'MEL'

In [6]:
Perfomance = pd.concat([Perfomance_1,Perfomance_2])

# Split inputs and feature Perfomance

In [7]:
Perfomance_inputs = Perfomance[Perfomance['Feature_Extract']=='RestNet50']
Perfomance_inputs = Perfomance_inputs.reset_index(drop=True)

In [8]:
Perfomance_inputs.groupby('Inputs').mean()

,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5
Inputs,,,,,,,,,,,,,
MEL,63.356940,0.99372,0.98084,0.99010,0.97390,0.99814,0.95649,0.99130,0.98869,0.98989,0.98807,1.000,0.998
MFCC,64.126715,0.95121,0.95084,0.97570,0.95947,0.98238,0.85742,0.95215,0.96607,0.89084,0.99080,0.955,0.981
SPEC,62.717188,0.94458,0.94550,0.93783,0.93871,0.97685,0.87222,0.95215,0.95911,0.89174,0.96055,0.966,1.000


In [9]:
Perfomance_Feature_Extract = Perfomance[Perfomance['Inputs']=='MEL']
Perfomance_Feature_Extract = Perfomance_Feature_Extract.reset_index(drop=True)

In [10]:
Perfomance_Feature_Extract.groupby('Feature_Extract').mean()

,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5
Feature_Extract,,,,,,,,,,,,,
INCEPTION_Resnet_v2,93.278855,0.89520,0.94163,0.89730,0.9045,0.92592,0.88242,0.90868,0.96346,0.80185,0.97892,0.946,0.981
RestNet50,63.356940,0.99372,0.98084,0.99010,0.9739,0.99814,0.95649,0.99130,0.98869,0.98989,0.98807,1.000,0.998
VGG16,59.860492,0.88178,0.91215,0.86847,0.9180,0.94998,0.77222,0.91651,0.93040,0.78163,0.96791,0.892,0.975


# Calculate by build in function

## inputs

In [11]:
confidence = pd.DataFrame(columns=["Inputs", 'Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high'])
confidence['Inputs'] = Perfomance_inputs['Inputs'].unique() 
for  k in Perfomance_inputs['Inputs'].unique():
    confidence.loc[confidence['Inputs'] ==  k,['Time-low','Time-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Time']), loc=np.mean(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Time']), scale=st.sem(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Time'])) 
    confidence.loc[confidence['Inputs'] ==  k,['Keyword-low','Keyword-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Command_Acc_p15']), loc=np.mean(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Command_Acc_p15']), scale=st.sem(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Command_Acc_p15'])) 
    confidence.loc[confidence['Inputs'] ==  k,['Speaker-low','Speaker-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Partcp_Acc_p15']), loc=np.mean(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Partcp_Acc_p15']), scale=st.sem(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Partcp_Acc_p15'])) 
confidence

,Inputs,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,MFCC,60.0986,68.1548,0.947122,0.954558,0.947686,0.954734
1,SPEC,59.2269,66.2075,0.940744,0.950256,0.940998,0.948162
2,MEL,59.0262,67.6877,0.979085,0.982595,0.992617,0.994823


In [12]:
confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']] = confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']].astype('float')

In [13]:
confidence.round(3)

,Inputs,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,MFCC,60.099,68.155,0.947,0.955,0.948,0.955
1,SPEC,59.227,66.207,0.941,0.950,0.941,0.948
2,MEL,59.026,67.688,0.979,0.983,0.993,0.995


## feature extraction

In [14]:
confidence = pd.DataFrame(columns=["Feature_Extract", 'Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high'])
confidence['Feature_Extract'] = Perfomance_Feature_Extract['Feature_Extract'].unique() 
for k in Perfomance_Feature_Extract['Feature_Extract'].unique():
    confidence.loc[confidence['Feature_Extract'] ==  k,['Time-low','Time-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Time']), loc=np.mean(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Time']), scale=st.sem(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Time'])) 
    confidence.loc[confidence['Feature_Extract'] ==  k,['Keyword-low','Keyword-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Command_Acc_p15']), loc=np.mean(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Command_Acc_p15']), scale=st.sem(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Command_Acc_p15'])) 
    confidence.loc[confidence['Feature_Extract'] ==  k,['Speaker-low','Speaker-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Partcp_Acc_p15']), loc=np.mean(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Partcp_Acc_p15']), scale=st.sem(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Partcp_Acc_p15'])) 
confidence

,Feature_Extract,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,RestNet50,59.0262,67.6877,0.979085,0.982595,0.992617,0.994823
1,VGG16,58.2777,61.4433,0.90726,0.91704,0.878281,0.885279
2,INCEPTION_Resnet_v2,87.8072,98.7505,0.93585,0.94741,0.888745,0.901655


In [15]:
confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']] = confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']].astype('float')

In [16]:
confidence.round(3)

,Feature_Extract,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,RestNet50,59.026,67.688,0.979,0.983,0.993,0.995
1,VGG16,58.278,61.443,0.907,0.917,0.878,0.885
2,INCEPTION_Resnet_v2,87.807,98.751,0.936,0.947,0.889,0.902


# calculate by hand - Inputs

In [17]:
confidence = pd.DataFrame(columns=["Inputs", 'Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_'])
confidence['Inputs'] = Perfomance_inputs['Inputs'].unique() 

In [18]:
t_value =  2.262 # df=10-1, alpha = 0.05, two tails
def CI_95(data):
    n=len(data)
    x_bar = data.mean()
    x_std = data.std()
    CI = t_value*x_std/np.sqrt(n)
    return x_bar, CI

In [19]:
for k in Perfomance_inputs['Inputs'].unique():
    confidence.loc[confidence['Inputs'] == k ,['Time_mean','Time+_']] = CI_95(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] == k ,'Time']) 
    confidence.loc[confidence['Inputs'] == k ,['Keyword_mean', 'Keyword+_']] = CI_95(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] == k ,'Command_Acc_p15']) 
    confidence.loc[confidence['Inputs'] == k ,['Speaker_mean','Speaker+_']] = CI_95(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] == k ,'Partcp_Acc_p15'])  
confidence

,Inputs,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,MFCC,64.1267,4.08932,0.95084,0.00377485,0.95121,0.0035774
1,SPEC,62.7172,3.54331,0.9455,0.00482843,0.94458,0.00363609
2,MEL,63.3569,4.39655,0.98084,0.0017818,0.99372,0.00111979


In [20]:
confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']] = confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']].astype('float')

In [21]:
confidence.round(4)

,Inputs,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,MFCC,64.1267,4.0893,0.9508,0.0038,0.9512,0.0036
1,SPEC,62.7172,3.5433,0.9455,0.0048,0.9446,0.0036
2,MEL,63.3569,4.3965,0.9808,0.0018,0.9937,0.0011


# calculate by hand - Feature Extraction

In [22]:
confidence = pd.DataFrame(columns=["Feature_Extract", 'Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_'])
confidence['Feature_Extract'] = Perfomance_Feature_Extract['Feature_Extract'].unique() 

In [23]:
t_value =  2.262 # df=10-1, alpha = 0.05, two tails
def CI_95(data):
    n=len(data)
    x_bar = data.mean()
    x_std = data.std()
    CI = t_value*x_std/np.sqrt(n)
    return x_bar, CI

In [24]:
for k in Perfomance_Feature_Extract['Feature_Extract'].unique():
    confidence.loc[confidence['Feature_Extract'] == k ,['Time_mean','Time+_']] = CI_95(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] == k ,'Time']) 
    confidence.loc[confidence['Feature_Extract'] == k ,['Keyword_mean', 'Keyword+_']] = CI_95(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] == k ,'Command_Acc_p15']) 
    confidence.loc[confidence['Feature_Extract'] == k ,['Speaker_mean','Speaker+_']] = CI_95(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] == k ,'Partcp_Acc_p15'])  
confidence

,Feature_Extract,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,RestNet50,63.3569,4.39655,0.98084,0.0017818,0.99372,0.00111979
1,VGG16,59.8605,1.60685,0.91215,0.00496442,0.88178,0.0035521
2,INCEPTION_Resnet_v2,93.2789,5.55484,0.94163,0.00586756,0.8952,0.00655269


In [25]:
confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']] = confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']].astype('float')

In [26]:
confidence.round(4)

,Feature_Extract,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,RestNet50,63.3569,4.3965,0.9808,0.0018,0.9937,0.0011
1,VGG16,59.8605,1.6069,0.9122,0.0050,0.8818,0.0036
2,INCEPTION_Resnet_v2,93.2789,5.5548,0.9416,0.0059,0.8952,0.0066
